In [1]:
import joblib
import pandas as pd
import colorama as col
from matplotlib import pyplot as plt
import ffn
import numpy as np
import datetime as dt
import fxcmpy
from tqdm import tqdm
import sys
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report, f1_score, roc_auc_score
sys.path.append('../') 
import pyttsx3
engine = pyttsx3.init()
from slack_sdk import WebClient
_slack_token = joblib.load('TOKENS/_slack_token.dag')
client = WebClient(token=_slack_token)
import time
import warnings
import talib
#from pycaret.regression import *
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [2]:
TIK = ['AUD','NZD','GBP','JPY','CHF','CAD','SEK','NOK','ILS','MXN','USD','EUR']
RATE = [0.776,0.721,1.3912,1/105.91,1/0.892,1/1.2681,1/8.2884,1/8.4261,1/3.2385,1/20.1564,1,1/1.21]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE

x = 'EUR/USD'
_period = 'm5'
_period2 = 'H1'
_ticker = x.replace('/','')
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2016-08-30' # stop the train and begin the test there '2016-08-31'
_stop = '2018-12-31' # stop the test there. After that, it is kept for oos
_last = '2021-04-19' # '2020-12-31'
_nb_bougie_exit = 5555555
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_target = 0.002
_exposure = 2
_sl = 0.001
_rate = df_ratefx.loc[x[4:],'rate']
_size = _cash_ini / df_ratefx.loc[x[:3],'rate']
_trigger_spread = 0.025


In [3]:
database = joblib.load('BASES/'+_ticker+'_'+_period)

In [4]:
def coinflip(df):
    df['Signal'] = np.random.randint(3, size=len(df))-1
    return(df)

def timerange1D(df):
    df['Date'] = df.index
    df['Date'] = df['Date'].dt.strftime(date_format='%Y-%m-%d')
    return(df.sort_index(axis=0))

def split_learn(df):
    """[Split the dtaframe in train/test/oos and reduce train and test to dataframe with signal!=0 only]

    Args:
        df ([pandas]): [the datframe to split already featured]
        _start ([date]): [beginning of the df]
        _mid ([date]): [stop of the train and beginning of the test]
        _stop ([date]): [stop of the test and beginning of the oos]
        _last ([date]): [end of the oos]
    """    
    df = df.dropna()
    df['Date'] = pd.to_datetime(df.Date)
    df_train = df[(df.Date>=_start)&(df.Date<=_mid)]
    df_test = df[(df.Date>_mid)&(df.Date<=_stop)]
    df_oos = df[(df.Date>_stop)&(df.Date <= _last)]
    df_train = df_train[df_train.Signal!=0]
    df_test = df_test[df_test.Signal!=0]
    return(df_train, df_test,df_oos)

def strategy(df,_window=40):

    df['RSI_'+str(2)] = talib.RSI(df.Close,2)
    df['EMA200'] = talib.EMA(df.Close,200)

    df['Window_High'] = df.Close.iloc[::-1].rolling(55).max().iloc[::-1]
    df['Window_Low'] = df.Close.iloc[::-1].rolling(55).min().iloc[::-1]
    df['Window_Long'] = df.Close + (df.Close * _target)
    df['Window_Short'] = df.Close - (df.Close * _target)
    df['Window_sl_Long'] = df.Close - (df.Close * _sl)
    df['Window_sl_Short'] = df.Close + (df.Close * _sl)

    ##### CONDITIONS LONG
    _condition_1 = (df['Window_Long'] >= df['Window_High']) & (df['Window_sl_Long'] <= df['Window_Low'])

    ##### CONDITIONS SHORT
    _condition_1_bar = (df['Window_Short'] <= df['Window_Low']) & (df['Window_sl_Short'] >= df['Window_High'])

    ##### 1 condition
    df['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))
    return(df.sort_index(axis=0))

def DagMaxBase(df):
    """[Generate future_max for scaling]

    Args:
        df ([pandas]): [Generate a new]
    """ 
    # max([abs(df_train[label]).max(),abs(df_train[label]).max()])
       
    features_max = pd.DataFrame()
    try:
        for label in df.drop(['Date','Symbol','Signal'],axis=1).columns:
            features_max.loc[0,label] = abs(df[label]).max()
    except:
        for label in df.drop(['Date','Symbol'],axis=1).columns:
            features_max.loc[0,label] = abs(df[label]).max()   
    
    joblib.dump(features_max,'VARS/'+_ticker+'/'+_ticker+'_MAX.dag')
    return(features_max)

def bt(price,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl,_save=0,_bt_report=0):
    engine = pyttsx3.init()

    print('Librairies imported\n')
    print('_bt_report : ',_bt_report)
    print('Verbose : ',_verbose)
    print('Save : ',_save)
    print('\n')

    engine.say("Backtesting in progress")
    engine.runAndWait()


    _t1 = dt.datetime.now()
    print('Début des opérations horodatée à',dt.datetime.now())

    _total = 0
    _cash = _cash_ini
    _pnl = 0
    _tracker = 0

    DATE = []
    CONTRACT = []
    OPEN_POZ = []
    CLOSE_POZ = []
    RATE_OPEN_POZ = []
    RATE_CLOSE_POZ = []
    PNL_LAT = []
    PNL_REAL = []
    TOTAL_OPEN = []
    TOTAL_CLOSE = []
    PRICE_BUY = []
    PRICE_SELL = []
    DER_POZ = []
    TOTAL_PNL_LAT = []
    TOTAL_PNL_REAL = []
    EXPO_MAX = []
    TRACKER = []
    TRADE_DURATION = []
    PRICE_BUY = []
    PRICE_SELL = []
    EQUITY = []
    CASH = []
    
    df_resultats = pd.DataFrame(index=['Equity','Nbre Winners','Nbre winners long','Nbre winners short','Nbre Loosers','Nbre loosers long','Nbre loosers short','Max lenght of trade','Min lenght of trade',\
        'Average lenght of trade','Cumul pnl'])
    
    print('\nChargement de la nouvelle base\n\n')

    engine.say("קדימה")
    engine.runAndWait()

    print(col.Fore.MAGENTA,'Le rate du ticker',x,'est à ',_rate,col.Style.RESET_ALL)

    price = price[(price.index >= _year_bottom) & (price.index <= _year_top)]
    print('Bases chargées')

    print('TETEL process effectué')

    print(col.Fore.CYAN,'ENTERING THE BACKTEST',col.Style.RESET_ALL)

    time.sleep(0.2)
        
    price = price.dropna()

    _position = 0
    _equity = 0
    _nbtransactions = 0
    backtest_graph = pd.DataFrame()

    _winner = 0
    _looser = 0
    _longwinner = 0
    _longlooser = 0
    _shortwinner = 0
    _shortlooser = 0
    _index_entry = 0
    
    _average_duration = 0
    _total = 0

    

    _open_buy = 0
    _open_sell = 0

    for i in tqdm(range(0,len(price))):

        _size = _cash_ini / df_ratefx.loc[x[:3],'rate']

        ##### POSITIONS EN L'AIR 
        if i >= (len(price)-1) and (_position == 1 or _position == -1) :

            if _position == -1:
                _position = 99
                _pnl = - (price.CloseAsk.iloc[i] - _price_sell_mean) * _size * _open_sell * _rate
                _total += _pnl
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                
                if _pnl > 0:
                    _winner += _open_sell
                    _longwinner+=_open_sell
                else:
                    _looser += _open_sell
                    _shortlooser +=_open_sell
                
                TRADE_DURATION.append(i - _index_entry)
                
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                    print(_open_sell,'position closed at',price.CloseAsk.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(-1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.CloseAsk.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_sell)
                PRICE_SELL = []
                _open_sell = 0
                continue

            if _position == 1:

                _position = 99
                _pnl = (price.CloseBid.iloc[i] - _price_buy_mean) * _size * _open_buy * _rate
                _total += _pnl
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                
                if _pnl > 0:
                    _winner += _open_buy
                    _longwinner +=_open_buy
                else:
                    _looser += _open_buy
                    _longlooser += _open_buy

                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                    print(_open_buy,'positions closed at',price.CloseBid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.CloseBid.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_buy) 
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                PRICE_BUY = []
                _open_buy = 0
                continue
        
        # SI PAS DE POSITION
        if _position == 0:
            # BUY SIGNAL
            if  price.Signal[i] == 1: 
                _pnl = 0
                _open_buy += 1
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                _position = 1
                _index_entry = i
                _tracker = price.index[i]
                _nbtransactions += 1
                price_buy = price.CloseAsk.iloc[i]
                PRICE_BUY.append(price_buy)
                _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    print('Position 1 bought at', price_buy,'(verification liste',PRICE_BUY[-1],')')

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(1)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(price_buy)
                RATE_CLOSE_POZ.append(0)
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_OPEN.append(1) 
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0)
                continue 

            # SELL SIGNAL
            elif price.Signal[i] == -1: 
                _pnl = 0
                _open_sell += 1
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                _index_entry = i
                _tracker = price.index[i]
                _position = -1
                _nbtransactions += 1
                price_sell = price.CloseBid.iloc[i]
                PRICE_SELL.append(price_sell)
                _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(-1)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(price_sell)
                RATE_CLOSE_POZ.append(0) 
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0)
                TOTAL_OPEN.append(1)
                continue

            else :
                _pnl = 0
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                PNL_LAT.append(0)
                PNL_REAL.append(0)
                continue
        
        # SI POSITION LONG
        elif _position == 1:

            ### RE_ENGAGE BUY ON VALID SIGNAL
            if price.Signal[i] == 1 and i - _index_entry < _nb_bougie_exit and _trigger_reengage == 1\
                 and _open_buy < _exposure :
                _pnl = 0
                _open_buy += 1
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                _position = 1
                _index_entry = i
                _tracker = price.index[i]
                _nbtransactions += 1
                price_buy = price.CloseAsk.iloc[i]
                PRICE_BUY.append(price_buy)
                _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    print('Position (REENG) 1 bought at', price_buy,'(verification liste',PRICE_BUY[-1],')')

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(1)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(price_buy)
                RATE_CLOSE_POZ.append(0)
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_OPEN.append(1) 
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0)
                continue

            ### CLOSE LONG ON INVERSE SIGNAL
            if price.Signal[i] == -1 and _trigger_invers == 1:
                _position = 0
                _pnl = (price.CloseBid.iloc[i] - _price_buy_mean) * _size * _open_buy * _rate
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                EXPO_MAX.append(_open_buy)
                CASH.append(_cash)
                if _pnl >=0:
                    _winner += _open_buy
                    _longwinner += _open_buy
                    TRACKER.append(_tracker)
                else:
                    _looser += _open_buy
                    _longlooser +=_open_buy

                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    if _pnl < 0:
                        print(_open_buy,'positions (INV) closed at',price.CloseBid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    else :
                        print(_open_buy,'positions (INV) closed at',price.CloseBid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.CloseBid.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_buy) 
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                PRICE_BUY = []
                _open_buy = 0
                continue
            
            ### CLOSE LONG ON TIME EXIT
            if i - _index_entry >= _nb_bougie_exit:
                _position = 0
                _pnl = (price.CloseBid.iloc[i] - _price_buy_mean) * _size * _open_buy * _rate
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                EXPO_MAX.append(_open_buy)
                CASH.append(_cash)
                if _pnl >=0:
                    _winner += _open_buy
                    _longwinner +=_open_buy
                    TRACKER.append(_tracker)
                else:
                    _looser += _open_buy
                    _longlooser += _open_buy

                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    if _pnl < 0:
                        print(_open_buy,'positions (TIME EXIT) closed at',price.CloseBid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    else :
                        print(_open_buy,'positions (TIME EXIT) closed at',price.CloseBid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.CloseBid.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_buy) 
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                PRICE_BUY = []
                _open_buy = 0
                continue
            
            # CLOSE LONG ON TARGET
            if (float(price.HighBid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) >= _target and _trigger_target == 1:
                _position = 0
                _pnl = (price.HighBid.iloc[i] - _price_buy_mean) * _size * _open_buy * _rate
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                EXPO_MAX.append(_open_buy)
                CASH.append(_cash)
                if _pnl >=0:
                    _winner += _open_buy
                    _longwinner += _open_buy
                    TRACKER.append(_tracker)
                else:
                    _looser += _open_buy
                    _longlooser += _open_buy

                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    if _pnl < 0:
                        print(_open_buy,'positions (TG) closed at',price.HighBid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    else :
                        print(_open_buy,'positions (TG) closed at',price.HighBid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.HighBid.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_buy) 
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                PRICE_BUY = []
                _open_buy = 0
                continue

            # CLOSE LONG ON STOP LOSS
            if (float(price. LowBid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) <= - _sl and _trigger_sl == 1:
                _position = 0
                _pnl = (price.LowBid.iloc[i] - _price_buy_mean) * _size * _open_buy * _rate
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                EXPO_MAX.append(_open_buy)
                CASH.append(_cash)
                if _pnl >=0:
                    _winner += _open_buy
                    _longwinner += _open_buy
                    TRACKER.append(_tracker)
                else:
                    _looser += _open_buy
                    _longlooser += _open_buy

                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    if _pnl < 0:
                        print(_open_buy,'positions (SL) closed at',price.LowBid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    else :
                        print(_open_buy,'positions (SL) closed at',price.LowBid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.LowBid.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_buy) 
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                PRICE_BUY = []
                _open_buy = 0
                continue
            
            else:

                _pnl = (price.CloseBid.iloc[i] - _price_buy_mean) * _size * _open_buy * _rate
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(0) 
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0)
                continue 
 
        # SI POSITION SHORT
        elif _position == -1:

            ### RE-ENGAGE SELL ON VALID SIGNAL
            if price.Signal[i] == -1 and i - _index_entry < _nb_bougie_exit and _trigger_reengage == 1 \
                and _open_sell < _exposure :
                
                _pnl = 0
                _open_sell += 1
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                _index_entry = i
                _tracker = price.index[i]
                _position = -1
                _nbtransactions += 1
                price_sell = price.CloseBid.iloc[i]
                PRICE_SELL.append(price_sell)
                _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    print('Position (REENG) 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(-1)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(price_sell)
                RATE_CLOSE_POZ.append(0) 
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0)
                TOTAL_OPEN.append(1)
                continue

            ### CLOSE SHORT ON INVERSE SIGNAL
            if price.Signal[i] == 1 and _trigger_invers == 1:   
                _position = 0
                _pnl = - (price.CloseAsk.iloc[i] - _price_sell_mean) * _size * _open_sell * _rate
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                EXPO_MAX.append(_open_sell)
                CASH.append(_cash)
                if _pnl >=0:
                    _winner += _open_sell
                    _shortwinner += _open_sell
                    TRACKER.append(_tracker)
                else:
                    _looser += _open_sell
                    _shortlooser += _open_sell
                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    if _pnl < 0 :    
                        print(_open_sell,'position (INV) closed at',price.CloseAsk.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    else:
                        print(_open_sell,'position (INV) closed at',price.CloseAsk.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(-1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.CloseAsk.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_sell)
                PRICE_SELL = []
                _open_sell = 0
                continue

            ### CLOSE SHORT ON TIME EXIT
            if i - _index_entry >= _nb_bougie_exit:   
                _position = 0
                _pnl = - (price.CloseAsk.iloc[i] - _price_sell_mean) * _size * _open_sell * _rate
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                EXPO_MAX.append(_open_sell)
                CASH.append(_cash)
                if _pnl >=0:
                    _winner += _open_sell
                    _shortwinner += _open_sell
                    TRACKER.append(_tracker)
                else:
                    _looser += _open_sell
                    _shortlooser += _open_sell
                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    if _pnl < 0 :    
                        print(_open_sell,'position (TIME EXIT) closed at',price.CloseAsk.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    else:
                        print(_open_sell,'position (TIME EXIT) closed at',price.CloseAsk.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(-1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.CloseAsk.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_sell)
                PRICE_SELL = []
                _open_sell = 0
                continue

            ### CLOSE SHORT ON TARGET
            if (float(price.LowAsk.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) <= -_target and _trigger_target == 1:
                _position = 0
                _pnl = - (price.LowAsk.iloc[i] - _price_sell_mean) * _size * _open_sell * _rate
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                EXPO_MAX.append(_open_sell)
                CASH.append(_cash)
                if _pnl >=0:
                    _winner += _open_sell
                    _shortwinner += _open_sell
                    TRACKER.append(_tracker)
                else:
                    _looser += _open_sell
                    _shortlooser +=_open_sell
                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    if _pnl < 0 :    
                        print(_open_sell,'position (TG) closed at',price.LowAsk.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    else:
                        print(_open_sell,'position (TG) closed at',price.LowAsk.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(-1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.LowAsk.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_sell)
                PRICE_SELL = []
                _open_sell = 0
                continue

            ### CLOSE SHORT ON STOP LOSS
            if (float(price.HighAsk.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) > _sl and _trigger_sl == 1:
                _position = 0
                _pnl = - (price.HighAsk.iloc[i] - _price_sell_mean) * _size * _open_sell * _rate
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                EXPO_MAX.append(_open_sell)
                CASH.append(_cash)
                if _pnl >=0:
                    _winner += _open_sell
                    _shortwinner += _open_sell
                    TRACKER.append(_tracker)
                else:
                    _looser += _open_sell
                    _shortlooser +=_open_sell
                TRADE_DURATION.append(i - _index_entry)
                if _verbose == 1:
                    print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                    if _pnl < 0 :    
                        print(_open_sell,'position (SL) closed at',price.HighAsk.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    else:
                        print(_open_sell,'position (SL) closed at',price.HighAsk.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(-1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.HighAsk.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_sell)
                PRICE_SELL = []
                _open_sell = 0
                continue

            else:

                _pnl = - (price.CloseAsk.iloc[i] - _price_sell_mean) * _size * _open_sell * _rate
                _equity = _cash + _pnl

                EQUITY.append(_equity)
                CASH.append(_cash)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(0)
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0)
                continue
        

    try:
        _average_duration = round(sum(TRADE_DURATION)/len(TRADE_DURATION),2)
        _max_duration = max(TRADE_DURATION)
        _min_duration = min([item for item in TRADE_DURATION if item !=0])

    except:
        print("(No Duration)") 
        _average_duration = 'NA'
        _max_duration = 0.00002
        _min_duration = 0.00001 
    print(col.Fore.BLUE,'For ticker',col.Fore.YELLOW,x,col.Style.RESET_ALL)
    if _total > 0:              
        print(col.Fore.MAGENTA,"\nTotal Profit & Loss : $",col.Fore.GREEN,round(_total,2),'. En ',\
            _nbtransactions,col.Style.RESET_ALL,' transactions.' )
    else:
        print(col.Fore.MAGENTA,"\nTotal Profit & Loss : $",col.Fore.RED,round(_total,2),'. En ',\
            _nbtransactions,col.Style.RESET_ALL,' transactions.' ) 
    print(col.Fore.GREEN,"\nWinners Number :",_winner,col.Style.RESET_ALL)
    print(col.Fore.RED,"\nLoosers number :",_looser,col.Style.RESET_ALL)

    backtest_graph['Equity'] = EQUITY

    df_resultats[x] = [(round(_equity,2)),(_winner),(_longwinner),(_shortwinner),(_looser),(_longlooser),(_shortlooser),(_max_duration),(_min_duration),(_average_duration),(_total)]

    DER_POZ.append(_pnl)

    engine.say("Finito caucau")
    engine.runAndWait()
    _t2 = dt.datetime.now()
    print("BT's execution time",str((_t2 - _t1)))
    df_historical = pd.DataFrame()
    df_historical = pd.DataFrame(index=DATE)
    df_historical['Contract'] = CONTRACT
    df_historical['Open_Poz'] = OPEN_POZ
    df_historical['Close_Pos'] = CLOSE_POZ
    df_historical['Rate_Open_Poz'] = RATE_OPEN_POZ 
    df_historical['Rate_Close_Poze'] = RATE_CLOSE_POZ
    df_historical['Pnl_Lat'] = TOTAL_PNL_LAT
    df_historical['Pnl_Real'] = TOTAL_PNL_REAL
    df_historical = df_historical.sort_index()
    _generated_cash = round(df_historical.Pnl_Real.sum(),2)
    _generated_cash_perc = round((_generated_cash / _cash_ini) * 100,2)
    print(col.Fore.YELLOW,x,col.Fore.BLUE,'results',col.Style.RESET_ALL)
    print(col.Fore.MAGENTA,'Tested Period',_year_bottom,' à',_year_top,col.Style.RESET_ALL)
    print(col.Fore.CYAN,'Total Number of trades',max([sum(TOTAL_OPEN),sum(TOTAL_CLOSE)]),col.Style.RESET_ALL)
    if _generated_cash <= 0:
        print('Started Cash :',_cash_ini)
        print('P&L in currency:',col.Fore.RED,str(_generated_cash)+'$',col.Style.RESET_ALL)
        print('P&L in %:',col.Fore.RED,str(_generated_cash_perc)+'%',col.Style.RESET_ALL)

    else:
        print('Started Cash :',_cash_ini)
        print('P&L  in currency:',col.Fore.GREEN,str(_generated_cash)+'$',col.Style.RESET_ALL)
        print('P&L in %:',col.Fore.GREEN,str(_generated_cash_perc)+'%',col.Style.RESET_ALL)

    print('Average trade duration',_average_duration)
    print('# Winners ',df_resultats.T['Nbre Winners'].sum())
    print('# Winners long ',df_resultats.T['Nbre winners long'].sum())
    print('# Winners short ',df_resultats.T['Nbre winners short'].sum())

    print('# Loosers ',df_resultats.T['Nbre Loosers'].sum())
    print('# Loosers  long',df_resultats.T['Nbre loosers long'].sum())
    print('# Loosers  short',df_resultats.T['Nbre loosers short'].sum())
    print('Cumulated gains',round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum(),2))
    print('Cumulated losses',round(df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2))
    print(col.Fore.BLUE,'PROFIT FACTOR : ',\
        abs(round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum()/df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2)),col.Style.RESET_ALL)
    try:
        print(col.Fore.CYAN,'Winners Ratio :',\
            round((df_resultats.T['Nbre Winners'].sum()*100)/(df_resultats.T['Nbre Loosers'].sum()+df_resultats.T['Nbre Winners'].sum()),2),\
                '%',col.Style.RESET_ALL)
    except:
        print(col.Fore.CYAN,'Winners Ratio  :None',col.Style.RESET_ALL)
    try:
        print('Average Winners',round(sum(list(filter(lambda x:  x > 0,PNL_REAL)))/len(list(filter(lambda x:  x > 0,PNL_REAL))),2))
        print('% Average Winners',round(sum(list(filter(lambda x:  x > 0,PNL_REAL)))/len(list(filter(lambda x:  x > 0,PNL_REAL))) * 100 / _cash_ini,2))
    except:
        print('No winner')
    try:
        print('Average Loosers',round(sum(list(filter(lambda x:  x < 0,PNL_REAL)))/len(list(filter(lambda x:  x < 0,PNL_REAL))),2))
        print('% Average Loosers',round(sum(list(filter(lambda x:  x < 0,PNL_REAL)))/len(list(filter(lambda x:  x < 0,PNL_REAL))) / _cash_ini * 100,2))
    except:
        print('No looser')
    try:
        print('Average pnl',round(sum(PNL_REAL)/sum(TOTAL_OPEN),2))
        print('% Average pnl',round((sum(PNL_REAL)/len(set(PNL_REAL))) / _cash_ini * 100,2))
    except:
        print('No trade')
    
    print('Number of opened trades',sum(TOTAL_OPEN))
    print('Number of closed trades',sum(TOTAL_CLOSE))
    print('Max Exposure',max(EXPO_MAX),'x ',_size,'= ',max(EXPO_MAX)*_size,'$')
    candle_feedback = pd.DataFrame(index = price.index)
    candle_feedback['Symbol'] = x
    candle_feedback['Size'] = _size
    candle_feedback[_ticker] = EQUITY
    print('\n_bt_report :',_bt_report,'\n')
    
    if _bt_report == 1:
        joblib.dump(candle_feedback,'BT/'+_ticker+'_candle_feedback_oos.dag')

    elif _bt_report == 2:
        joblib.dump(candle_feedback,'BT/'+_ticker+'_candle_feedback_bt_bt.dag')
    
    elif _bt_report == 3:
        joblib.dump(candle_feedback,'BT/'+_ticker+'_candle_feedback_ai.dag')
    
    elif _bt_report == 3:
        joblib.dump(candle_feedback,'BT/'+_ticker+'_candle_feedback_bt.dag')
        
    return(TRACKER,df_resultats.T['Nbre Loosers'].sum())

In [5]:
%%time
database = strategy(database)
#database = coinflip(database)
_verbose = 0
_bt_report = 1
_year_bottom = _stop
_year_top = _last
print(col.Fore.RED,'###############################################################################################')
print(' ####################################### OOS WITHOUT AI ########################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
##### Backtest Over Night
TRACKER,_nb_looser = bt(database,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl,_bt_report=_bt_report)

print(col.Fore.CYAN+'\n\nTRAITEMENT STATISTIQUE DES DONNEES\n',col.Style.RESET_ALL)
candle_feedback = joblib.load('BT/'+_ticker+'_candle_feedback_oos.dag')
candle_feedback = timerange1D(candle_feedback)
stats = candle_feedback.drop(['Symbol','Size','Date'],axis=1).calc_stats()
stats[_ticker].display()
print(col.Fore.CYAN+'\n\nMONTHLY RETURN FOR '+x+'\n',col.Style.RESET_ALL)
stats[_ticker].display_monthly_returns()
_bt_report = 0

_year_bottom = _start
_year_top = _stop
print()
print(col.Fore.CYAN,'###############################################################################################')
print(' #################################### TRAIN/TEST WITHOUT AI ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)


TRACKER,_nb_looser = bt(database,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.BLUE,'###############################################################################################')
print(' #################################### DENOISING & ENHANCING ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
while _nb_looser > 0 :
    
    database['TRACKER'] = np.where(database.index.isin(TRACKER),1,0)
    database['Valid'] = np.where(((database.Signal!=0)&(database.TRACKER==1)),1,0)
    database['Signal'] = np.where(((database.Valid==1)&(database.Signal==1)),1,np.where(((database.Valid==1)&(database.Signal==-1)),-1,0))
    ##### Purification of signal by denoising and enhancing
    TRACKER,_nb_looser = bt(database,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl)

print("OK")

 ###############################################################################################
 ####################################### OOS WITHOUT AI ########################################
 ############################################################################################### 
Librairies imported

_bt_report :  1
Verbose :  0
Save :  0


Début des opérations horodatée à 2021-04-27 21:44:14.223761

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1.0 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 166297/166297 [00:12<00:00, 13753.02it/s]
 For ticker  EUR/USD 
 
Total Profit & Loss : $  146489.86 . En  1557   transactions.
 
Winners Number : 698 
 
Loosers number : 859 
BT's execution time 0:00:19.087978
 EUR/USD  results 
 Tested Period 2018-12-31  à 2021-04-19 
 Total Number of trades 1557 
Started Cash : 200000
P&L  in currency:  146489.86$ 
P&L in %:  73.24% 
Average trade duration 91.23
# Winners  698.0
# Winners long 

In [6]:
def featuring(df):
    features = pd.DataFrame()
    features = pd.DataFrame(index=df.index)
    features['Symbol'] = df['Symbol']
    features['Date'] = df['Date']
    features['Open'] = df['Open']
    features['High'] = df['High']
    features['Low'] = df['Low']
    features['Close'] = df['Close']
    features['FEMA_21'] = df['Close'] * df['EMA_21'] # 0.76 en *, 0.48 en + et 0.35 en -
    features['FEMA_8'] = df['Close'] * df['EMA_8']# 1.31 en *
    features['FADRLo'] = df['Close'] * df['ADR_Low'] # 1.38 
    features['FADRHi'] = df['Close'] * df['ADR_High'] # 1.59 
    features['FRVI40'] = df['Close'] * df['RVI'] # 0.21 en *, 0.07 seul et 0 avec *df['Close']/100
    features['FRVI60'] = df['Close'] * df['RVI'] # 0.21 en *, 0.07 seul et 0 avec *df['Close']/100
    features['FONLOSMA5'] = df['Close'] * df['ONLOSMA_5'] # 0.9 en *, 0.14 seul et 0 avec df['Low'] - 
    features['FONHISMA5'] = df['Close'] * df['ONHISMA_5'] # 0.97 en *, 0.41 seul et  0.07 avec df['High'] - 
    features['FONLOSMA21'] = df['Close'] * df['ONLOSMA_21'] # 0.35 en *, 0.28 seul et 0 avec df['Low'] - 
    features['FONHISMA21'] = df['Close'] * df['ONHISMA_21'] # 1.04 en *, 0.55 seul et 0.28 avec df['High'] - 
    features['FONLOSMA34'] = df['Close'] * df['ONLOSMA_34'] # 0.55 en *, 0.35 seul et 0 avec df['Low'] - 
    features['FONHISMA34'] = df['High'] * df['ONHISMA_34'] # 0.83 en *, 0.48 en + 0.14 avec et 0 sans
    features['FSBGAMMA'] = df['SB_Gamma'] + df['Close'] # 0.14 avec + et 0 sans * df['Close']
    features['FOPENWEEKLY'] = df['Close'] * df['OpenShift_weekly'] # 0.55 en * et 0.28 en +
    features['FHIGHWEEKLY'] = df['Close'] * df['HighShift_weekly'] # 0.48 en * et 0 en -
    features['FLOWWEEKLY'] = df['Close'] * df['LowShift_weekly'] # 0.35 en * et 0.07 en -
    features['FCLOSEWEEKLY'] = df['Close'] * df['CloseShift_weekly'] # 0.48 en * et 0 en -
    features['FOPENDAILY'] = df['Close'] * df['OpenShift_daily'] # 0.14 en * et 0.14 en -
    features['FHIGHDAILY'] = df['Close'] * df['HighShift_daily'] # 1.24 en * et 0 en -
    features['FLOWDAILY'] = df['Close'] * df['LowShift_daily'] # 0.69 en * et 0 en -
    features['FCLOSEDAILY'] = df['Close'] * df['CloseShift_daily'] # 1.24 en * et 0 en -
    '''#features['FOPENHOURLY'] = df['Close'] - df['OpenShift_hourly']#.shift(1)
    #features['FHIGHHOURLY'] = df['Close'] - df['HighShift_hourly']#.shift(1)
    #features['FLOWHOURLY'] = df['Close'] - df['LowShift_hourly']#.shift(1)
    #features['FCLOSEHOURLY'] = df['Close'] - df['CloseShift_hourly']#.shift(1)'''
    features['FSMA200'] = (df['Close'] * df['SMA_200']) # 0.41 en * et 0 en -
    features['FBOLUP20'] = (df['Close'] * df['UpperBand']) # 0.55 en * et 0.21 en -
    features['FBOLLOW20'] = (df['Close'] * df['LowerBand']) # 0.14 en * et 0.21 en -
    features['FRSI2H'] = df['Close'] - df['RSI_2_hourly'] # 0.0 en * et 0.14 en -
    features['FRSI14H'] = df['Close'] - df['RSI_14_hourly'] # 0.07 en * et 0.41 en -
    features['FPP'] = df['Close'] * df['PP'] # 0.35 en * et 0 en -
    features['FS38'] = df['Close'] * df['S38'] # 0.28 en * et 0 en -
    features['FS62'] = df['Close'] * df['S62'] # 0.48 en * et 0 en -
    features['FS78'] = df['Close'] * df['S78'] # 0.28 en * et 0 en -
    features['FS100'] = df['Close'] * df['S100'] # 0.14 en * et 0.07 en -
    features['FS138'] = df['Close'] + df['S138'] # 0.35 en + et 0 en -
    features['FS162'] = df['Close'] * df['S162'] # 0.07 en * et 0 en -
    features['FS178'] = df['Close'] * df['S178'] # 0.07 en * et 0 en -
    features['FS200'] = df['Close'] * df['S200'] # 0.41 en * ou + et 0 en -
    features['FR38'] = df['Close'] * df['R38'] # 1.38 en *, 1.31 en + et 0 en -
    features['FR62'] = df['Close'] * df['R62'] # 0.41 en * et 0 en -
    features['FR78'] = df['Close'] * df['R78'] # 1.17 en +, 0.41 en * et 0 en -
    features['FR100'] = df['Close'] + df['R100'] # 1.66 en +, 1.31 en * et 0.28 en -
    features['FR138'] = df['Close'] + df['R138'] # 2.14 en +, 2.07 en * et 0.14 en -
    features['FR162'] = df['Close'] + df['R162'] # 2.9 en +, 2.28 en * et 0.35 en -
    features['FR178'] = df['Close'] + df['R178']# 2.7 en +, 2.14 en * et 0.28 en -
    features['FR200'] = df['Close'] + df['R200']# 2.97 en +, 2.83 en * et 0.07 en -
    features['SBATR'] = (df['Close'] - df['Open']) / df['ATR_14'] # 0.21
    features['FRSI2'] = df['Close'] + df['RSI_2_hourly'] # 0.41 en +, 0.14 avec *df['Close']/100
    features['FRSI14'] = df['Close'] + df['RSI_14_hourly'] # 0.28 avec + 0.07 avec *df['Close']/100
    features['Signal'] = df['Signal']
    
    #features = features.dropna()
    return(features.sort_index(axis=0))

In [7]:
%%time
database = strategy(database)
#database = coinflip(database)

features = featuring(database)
print('Shape de features',features.shape)
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# On établit les constantes et on les sauvegarde
features_max = DagMaxBase(features)

# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

#features_train, features_test, features_oos = split_learn(features)

#print('Shape de features',features.shape,features_test.shape,features_oos.shape)


Shape de features (820140, 53)
CPU times: user 1.44 s, sys: 681 ms, total: 2.12 s
Wall time: 1.65 s


df = features.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1).copy()

# full correlation table
df.corr().style.background_gradient(cmap='viridis')

df = df[['Close', 'FRVI40', 'FRVI60', 'FSBGAMMA', 'FRSI2H',	'FRSI14H' ,'SBATR',	'FRSI2', 'FRSI14' , 'FS78' ,'FR78']]
# full correlation table
df.corr().style.background_gradient(cmap='viridis')

%%time
# outlier check
plt.figure(figsize=(15, 20))

for i, c in enumerate(df.select_dtypes(include='number').columns):
    plt.subplot(12,2,i*2+1)
    sns.boxplot(df[c], color='blue')
    plt.title('Distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

    
    plt.subplot(12,2,i*2+2)
    sns.boxplot(df[c].apply('log1p'), color='red')
    plt.title('Log1p distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

%%time
plt.figure(figsize=(15, 12))

for i, c in enumerate(df.select_dtypes(include='number').columns):
    plt.subplot(12,2,i+1)
    sns.distplot(df[c])
    plt.title('Distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

%%time
skewed_cols = ['SBATR','FSBGAMMA']
# Apply log1p transformation on dataframe - just selected values
for c in skewed_cols:
    df[c] = df[c].apply('log1p')
# Next check & fix strongly skewed features
# apply log1p transform
plt.figure(figsize=(15, 12))

for i, c in enumerate(skewed_cols):
    plt.subplot(5,2,i+1)
    sns.distplot(df[c].apply(np.log1p))
    plt.title('Distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)


In [8]:
%%time

#features = features[['Date','Symbol','Close', 'FRVI40', 'FRVI60', 'FSBGAMMA', 'FRSI2H',	'FRSI14H' ,'SBATR',	'FRSI2', 'FRSI14' , 'FS78' ,'FR78','Signal','TRACKER','Valid']]
features_oos = features.iloc[int(0.8*len(features)):,:]

df_oos = database.iloc[int(0.8*len(features)):,:]

features = features.iloc[:int(0.8*len(features)),:]
features = features[features.Signal != 0]
features['Signal'] = np.where(features.Valid==1,1,0)



CPU times: user 64.9 ms, sys: 20.6 ms, total: 85.5 ms
Wall time: 81 ms


In [9]:
%%time

for label in features.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1).columns:
    features[label] = features[label] / features_max[label][0]



CPU times: user 158 ms, sys: 52 ms, total: 210 ms
Wall time: 150 ms


In [10]:
from sklearn.utils import resample
features.Signal.value_counts()

0    290788
1      2636
Name: Signal, dtype: int64

In [11]:
3150/ 21779

0.14463473988704714

In [12]:
# Split data on majority and minority.. minority is dataset == 2
minority = features[features.Signal==1]
majority = features[features.Signal==0]

print('Minority size:', minority.shape)
print('Majority size:', majority.shape)

Minority size: (2636, 55)
Majority size: (290788, 55)


In [13]:
# choosing upsample as even now we do not have too much data
minority_downsample = resample(majority, replace=True, n_samples=minority.shape[0])
print('Majority downsampled size:', minority_downsample.shape)

Majority downsampled size: (2636, 55)


In [14]:
features = pd.concat([minority_downsample, minority], axis=0)
features.shape

(5272, 55)

In [15]:
%%time
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1), features.Valid, test_size=0.25, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)


Train values shape: (3954, 50)
Test values shape: (1318, 50)
Train target shape: (3954,)
Test target shape: (1318,)
CPU times: user 4.36 ms, sys: 1.62 ms, total: 5.99 ms
Wall time: 4.81 ms


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [17]:
%%time
# Logistic Regression
model1 = LogisticRegression()
model1.fit(X_train, y_train)
y_train_hat = model1.predict(X_train)
y_test_hat = model1.predict(X_test)

print(model1)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))

LogisticRegression()
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.62      0.67      0.64      2000
           1       0.63      0.58      0.60      1954

    accuracy                           0.62      3954
   macro avg       0.63      0.62      0.62      3954
weighted avg       0.63      0.62      0.62      3954

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.60      0.67      0.63       636
           1       0.65      0.59      0.62       682

    accuracy                           0.63      1318
   macro avg       0.63      0.63      0.63      1318
weighted avg       0.63      0.63      0.63      1318

Roc_auc score
-------------------------------------------------------
0.6272685774359542

Confusion matrix
-------------------------------------------------------
[[423 213]
 [28

In [18]:
%%time
# Support Vector Machines
model2 = SVC()
model2.fit(X_train, y_train)
y_train_hat = model2.predict(X_train)
y_test_hat = model2.predict(X_test)

print(model2)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))

SVC()
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.64      0.70      0.67      2000
           1       0.66      0.59      0.62      1954

    accuracy                           0.65      3954
   macro avg       0.65      0.65      0.64      3954
weighted avg       0.65      0.65      0.64      3954

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.59      0.68      0.63       636
           1       0.66      0.56      0.60       682

    accuracy                           0.62      1318
   macro avg       0.62      0.62      0.62      1318
weighted avg       0.62      0.62      0.62      1318

Roc_auc score
-------------------------------------------------------
0.6220397831018646

Confusion matrix
-------------------------------------------------------
[[435 201]
 [300 382]]
CPU tim

In [19]:
%%time
# Random Forest
_param3 = {'criterion': 'gini', 'max_leaf_nodes': 50, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
model3 = RandomForestClassifier(
criterion = 'gini',
max_leaf_nodes = 50,
min_samples_leaf =  2,
min_samples_split = 5,
n_estimators = 200)
    
    
# n_jobs=-1,random_state=123)
model3.fit(X_train, y_train)
y_train_hat = model3.predict(X_train)
y_test_hat = model3.predict(X_test)

print(model3)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))

RandomForestClassifier(max_leaf_nodes=50, min_samples_leaf=2,
                       min_samples_split=5, n_estimators=200)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      2000
           1       0.76      0.78      0.77      1954

    accuracy                           0.77      3954
   macro avg       0.77      0.77      0.77      3954
weighted avg       0.77      0.77      0.77      3954

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.64      0.65      0.65       636
           1       0.67      0.66      0.67       682

    accuracy                           0.66      1318
   macro avg       0.66      0.66      0.66      1318
weighted avg       0.66      0.66      0.66      1318

Roc_auc score
-------------------------------------------------------
0.657

In [20]:
%%time
# Neural nets
model4 = MLPClassifier()
model4.fit(X_train, y_train)
y_train_hat = model4.predict(X_train)
y_test_hat = model4.predict(X_test)

print(model4)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))

MLPClassifier()
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.70      0.62      0.66      2000
           1       0.65      0.73      0.69      1954

    accuracy                           0.67      3954
   macro avg       0.68      0.68      0.67      3954
weighted avg       0.68      0.67      0.67      3954

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.69      0.62      0.65       636
           1       0.67      0.74      0.71       682

    accuracy                           0.68      1318
   macro avg       0.68      0.68      0.68      1318
weighted avg       0.68      0.68      0.68      1318

Roc_auc score
-------------------------------------------------------
0.6784107047345027

Confusion matrix
-------------------------------------------------------
[[392 244]
 [177 505

In [21]:
%%time
# K-Neighbors
model5 = KNeighborsClassifier()
model5.fit(X_train, y_train)
y_train_hat = model5.predict(X_train)
y_test_hat = model5.predict(X_test)
print(model5)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))

KNeighborsClassifier()
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.73      0.80      0.76      2000
           1       0.77      0.70      0.74      1954

    accuracy                           0.75      3954
   macro avg       0.75      0.75      0.75      3954
weighted avg       0.75      0.75      0.75      3954

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.55      0.65      0.59       636
           1       0.60      0.49      0.54       682

    accuracy                           0.57      1318
   macro avg       0.57      0.57      0.57      1318
weighted avg       0.58      0.57      0.57      1318

Roc_auc score
-------------------------------------------------------
0.573325310315572

Confusion matrix
-------------------------------------------------------
[[415 221]
 [3

In [22]:
%%time
from xgboost import XGBClassifier
model6 = XGBClassifier(
base_score = 0.5,
booster =  'gbtree',
gamma = 1,
learning_rate = 0.01,
n_estimators = 200,
reg_alpha = 0.5,
reg_lambda = 5,
random_state = 42
) # random_state=123)'''
model6.fit(X_train, y_train)
y_train_hat = model6.predict(X_train)
y_test_hat = model6.predict(X_test)

print(model6)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))

[21:47:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=16, num_parallel_tree=1, random_state=42,
              reg_alpha=0.5, reg_lambda=5, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

          

In [23]:
%%time
# Extra Trees
from sklearn.ensemble import ExtraTreesClassifier

model7 = ExtraTreesClassifier(random_state=123)
model7.fit(X_train, y_train)
y_train_hat = model7.predict(X_train)
y_test_hat = model7.predict(X_test)

print(model7)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))


ExtraTreesClassifier(random_state=123)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000
           1       1.00      1.00      1.00      1954

    accuracy                           1.00      3954
   macro avg       1.00      1.00      1.00      3954
weighted avg       1.00      1.00      1.00      3954

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.59      0.65      0.62       636
           1       0.64      0.58      0.61       682

    accuracy                           0.61      1318
   macro avg       0.62      0.62      0.61      1318
weighted avg       0.62      0.61      0.61      1318

Roc_auc score
-------------------------------------------------------
0.6157942787583688

Confusion matrix
-----------------------------------------------------

In [24]:
model8 = MLPClassifier(hidden_layer_sizes=(400,130), activation='relu', solver='adam', alpha=1e-06, batch_size='auto', learning_rate='constant', learning_rate_init=1e-05, power_t=0.5, max_iter=4100, shuffle=True, random_state=42, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)

model8.fit(X_train, y_train)
y_train_hat = model8.predict(X_train)
y_test_hat = model8.predict(X_test)

print(model8)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))


MLPClassifier(alpha=1e-06, beta_1=0.8, hidden_layer_sizes=(400, 130),
              learning_rate_init=1e-05, max_iter=4100, random_state=42,
              tol=1e-07, warm_start=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.65      0.74      0.70      2000
           1       0.69      0.60      0.64      1954

    accuracy                           0.67      3954
   macro avg       0.67      0.67      0.67      3954
weighted avg       0.67      0.67      0.67      3954

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           0       0.62      0.73      0.67       636
           1       0.70      0.58      0.64       682

    accuracy                           0.66      1318
   macro avg       0.66      0.66      0.65      1318
weighted avg       0.66      0.66      0.65      1318

Roc_auc score
-

%%time
from sklearn.model_selection import GridSearchCV, KFold
# Extra Tree
# n_jobs=-1 to allow run it on all cores
params = {
    'n_estimators': [100, 200, 500],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [1,2,4,5],
    'min_samples_leaf': [1,2,4,5],
    'max_leaf_nodes': [4,10,20,50,None]
}

gs3 = GridSearchCV(ExtraTreesClassifier(n_jobs=-1), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
gs3.fit(X_train, y_train)

print('Best score:', gs3.best_score_)
print('Best score:', gs3.best_params_)

%%time
# Random forest
# n_jobs=-1 to allow run it on all cores
params = {
    'n_estimators': [100, 200, 500],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [1,2,4,5],
    'min_samples_leaf': [1,2,4,5],
    'max_leaf_nodes': [4,10,20,50,None]
}

gs1 = GridSearchCV(RandomForestClassifier(n_jobs=-1), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
gs1.fit(X_train, y_train)

print('Best score:', gs1.best_score_)
print('Best score:', gs1.best_params_)

%%time
# XGBoost
# n_jobs=-1 to allow run it on all cores
params = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01,0.05,0.1],
    'booster': ['gbtree', 'gblinear'],
    'gamma': [0, 0.5, 1],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [0.5, 1, 5],
    'base_score': [0.2, 0.5, 1]
}

gs2 = GridSearchCV(XGBClassifier(n_jobs=-1), params, n_jobs=-1, cv=KFold(n_splits=3), scoring='roc_auc')
gs2.fit(X_train, y_train)

print('Best score:', gs2.best_score_)
print('Best score:', gs2.best_params_)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier

# votes = [
#     ('rf', RandomForestClassifier(n_jobs=-1, criterion='gini', max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100)),
#     ('xgb', XGBClassifier(n_jobs=-1, base_score=0.2, booster='gbtree', gamma=0, learning_rate=0.1, n_estimators=500, reg_alpha=0, reg_lambda=1)),
#     ('xt', ExtraTreesClassifier(n_jobs=-1, criterion='gini', max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2, n_estimators=500))
# ]

votes = [
    ('rf', gs1.best_estimator_),
    ('xgb', gs2.best_estimator_),
    ('xt', gs3.best_estimator_)
]

# soft voting based on weights
votesClass = VotingClassifier(estimators=votes, voting='soft', n_jobs=-1)
votesClass_cv = cross_validate(votesClass, X_train, y_train, cv=KFold(3, random_state=123))
votesClass.fit(X_train, y_train)

votesClass_cv

from sklearn.ensemble import ExtraTreesClassifier

model = votesClass
#model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))

0

In [26]:
%%time

_model = model6

features_oos = featuring(df_oos)

for label in features_oos.drop(['Date','Symbol','Signal'],axis=1).columns:
    features_oos[label] = features_oos[label] / features_max[label][0]

#features_oos = features_oos[['Date','Symbol','Close', 'FRVI40', 'FRVI60', 'FSBGAMMA', 'FRSI2H',	'FRSI14H' ,'SBATR',	'FRSI2', 'FRSI14' , 'FS78' ,'FR78','Signal']]

df_oos = strategy(df_oos)
#database = coinflip(database)

df_oos['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal'],axis=1))


df_oos['Signal'] = np.where((df_oos.Signal==1)&(df_oos.Valid==1),1,np.where((df_oos.Signal==-1)&(df_oos.Valid==1),-1,0))


##### On backtest selon le ticker selectionné sur la période déterminée

print(x)
_year_bottom = _stop
_year_top = _last

_target = 0.002
_sl = 0.001
_nb_bougie_exit = 5555555

print()
print(col.Fore.GREEN,'###############################################################################################')
print(' ########################################### OOS AFTER AI ######################################')
print(' ###############################################################################################',col.Style.RESET_ALL)

##### Backtest Over Night
_bt_report = 3
FINAL_TRACKER = bt(df_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl,_bt_report=_bt_report)

print(col.Fore.CYAN+'\n\nTRAITEMENT STATISTIQUE DES DONNEES\n',col.Style.RESET_ALL)
candle_feedback = joblib.load('BT/'+_ticker+'_candle_feedback_ai.dag')
candle_feedback = timerange1D(candle_feedback)
stats = candle_feedback.drop(['Symbol','Size','Date'],axis=1).calc_stats()
stats[_ticker].display()
print(col.Fore.CYAN+'\n\nMONTHLY RETURN FOR '+x+'\n',col.Style.RESET_ALL)
stats[_ticker].display_monthly_returns()
_bt_report = 0


EUR/USD

 ###############################################################################################
 ########################################### OOS AFTER AI ######################################
 ############################################################################################### 
Librairies imported

_bt_report :  3
Verbose :  0
Save :  0


Début des opérations horodatée à 2021-04-27 21:51:40.836318

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1.0 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 166297/166297 [00:13<00:00, 12361.34it/s]
 For ticker  EUR/USD 
 
Total Profit & Loss : $  -232726.56 . En  4063   transactions.
 
Winners Number : 1214 
 
Loosers number : 2849 
BT's execution time 0:00:19.873773
 EUR/USD  results 
 Tested Period 2018-12-31  à 2021-04-19 
 Total Number of trades 4063 
Started Cash : 200000
P&L in currency:  -232726.56$ 
P&L in %:  -116.36% 
Average trade duration 38.34
# Winners  1214.0


In [34]:
df_oos.shape,features_oos.shape

((248878, 65), (248878, 14))

In [48]:
features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17899 entries, 2015-09-22 08:10:00 to 2014-09-29 15:35:00
Data columns (total 55 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Symbol        17899 non-null  object        
 1   Date          17899 non-null  datetime64[ns]
 2   Open          17899 non-null  float64       
 3   High          17899 non-null  float64       
 4   Low           17899 non-null  float64       
 5   Close         17899 non-null  float64       
 6   FEMA_21       17899 non-null  float64       
 7   FEMA_8        17899 non-null  float64       
 8   FADRLo        17899 non-null  float64       
 9   FADRHi        17899 non-null  float64       
 10  FRVI40        17899 non-null  float64       
 11  FRVI60        17899 non-null  float64       
 12  FONLOSMA5     17899 non-null  float64       
 13  FONHISMA5     17899 non-null  float64       
 14  FONLOSMA21    17899 non-null  float64       
 15  F